In [ ]:
# noticing that it improves to a point and then get bad
# try decreasing learning rate of sl (less overfitting slower learning more convergence?)
#   try 0.000125 (defualt 0.0005)
# try increasing rl learning rate (maybe to 0.005), and maybe try increasing sl learning rate a tiny bit now that i have decreased replay interval (and we are hopefully getting less repeated experiences)
# try changing buffer size of sl (smaller more convergence? bigger less forgetting?)
#   try 125000 (default 250000)
#   try 500000
# try changing number of filters of sl (not sure how this would help with convergence but maybe forgetting)
#   try 128 (default 64)
# try lowering replay interval? to decrease change between training steps and repeat experiences
#   try 64 and 32 and 16? (default 128)
# try increasing replay interval for faster convergence?
#   try 256?
# try decreasing minibatches to stop overfitting?
#   try 2 and 1? (default 4)

# noticing rl agent gets negative exploitability sometimes, means something is wrong with rl agent
# maybe not long enough replay memory? so it tries something its already tried that is bad?
# maybe not enough filters?
# maybe overfitting? learning rate too high? batch size too small?


# config 4 negative rl agent vs p1, may not have let it train long enough.
# config 5 super super slow training time (12 minutes per 1000 iterations compared to usual 1.5 minutes)
# config 6 seemingly very good but only plays 0 on first move (probably only plays 1 move on first move) and never changes, so doesnt seem to explore enough. RL agent does not have accurate q value (probably lack of playing and seeing those moves). SO OVERFIT TO ONE SET IT DOESNT DRAW WITH ANY OTHER MOVES, even after first same move
# config 7 some negatives and same problems as 6 (i just decreased filters)
# config 8 has negatives
# config 16 starts good, noticing all the rl agents have very optimistic value predictions (which is really good). seems to have stopped improving after continuing training, may be a bug in loading/saving
# config 16 with eg epsilon and 500k was good, maybe try higher eg? or higher noisy sigma? maybe  try using some stuff from 6

# config 4 -> 5: changed to not shared_networks
# config 5 -> 6: switched to 16 filters of conv layers from 64 3x3 and 64 2x2 res layers
# config 6 -> 7: switched to 8 filters of conv layers and 2M sl memory
# config 7 -> 8: switched to 16 filters of conv layers
# config 8 -> 16: switched to 64 filters of conv layers, (still no eg_epsilon, even though i set it final step is 0), switched to lr 0.0025 and Adam optimizer for RL, noisy sigma 0.06, 4 minibatches, RL buffer 200000, replay interval down to 32, shared networks true, and added weight decay, only 100K training steps
# *** all trials before 16 had non linear per beta annealing, 16 switched to linear after 100K steps were non linear
# config 16 -> 31: switched to adam and adam, added eg_epsilon linear, and noisy sigma 0.5, 500K training steps, 2 minibatches instead of 4,
# config 16 -> 32: switched to 0.5 noisy sigma, removed dense layers,
# config 32 -> 33: thinking of removing clip norm, removed weight decay, maybe adding more eg with linear decay? so in later positions rl explores more? also would like to maybe for 34 try dense layers and see if they are better, also try double SGD and double Adam with both conv and dense so those would be 35 and 36


# 32 was very good overall, needed more training time, and never trained on move 3 (though still preformed well)

# try 16 with some low prob clipping
# try with some both adam or both SGD

# ALL ABOVE AGENTS HAD A DENSE LAYER OF 128

In [1]:
from nfsp_agent_clean import NFSPDQN
from agent_configs import NFSPDQNConfig, RainbowConfig
from game_configs import MississippiMarblesConfig, LeducHoldemConfig, TicTacToeConfig
from utils import KLDivergenceLoss, CategoricalCrossentropyLoss, HuberLoss, MSELoss
from torch.optim import Adam, SGD

config_dict = {
    "shared_networks_and_buffers": False,
    "training_steps": 7.5e5,  # like 2-5M in the paper (1M for initial test and see if rainbow is faster sooner)
    "num_players": 2,
    "anticipatory_param": 0.1,
    "replay_interval": 128,
    "num_minibatches": 2,  # 4 <- 16
    "learning_rate": 0.1,  # 0.1 with SGD
    # "weight_decay": 1e-9,
    # "clipnorm": 1.0,
    "optimizer": SGD,  # SGD
    "loss_function": MSELoss(),
    # "per_epsilon": 1e-3,
    "min_replay_buffer_size": 128,
    "minibatch_size": 128,
    "replay_buffer_size": 200000,  # 50000
    "transfer_interval": 300,
    "residual_layers": [],
    "conv_layers": [],  # [(32, 32, 1)]
    "dense_layer_widths": [64, 64, 64],  # [], [512]
    "value_hidden_layer_widths": [],
    "advantage_hidden_layer_widths": [],
    "noisy_sigma": 0.0,  # 0.06
    "eg_epsilon": 0.06,  # 0.1, 0 <- 16, 0.06
    "eg_epsilon_decay_type": "inverse_sqrt",  # linear, inverse_sqrt
    "eg_epsilon_decay_final_step": 500000,
    "sl_learning_rate": 0.005,  # 0.005 with SGD <- 16, 0.0005 with Adam, 0.000125 with Adam,
    # "sl_weight_decay": 1e-9,
    # "sl_clipnorm": 1.0,
    "sl_optimizer": SGD,  # SGD <- 16
    "sl_loss_function": CategoricalCrossentropyLoss(),
    "sl_min_replay_buffer_size": 128,
    "sl_minibatch_size": 128,
    "sl_replay_buffer_size": 2000000,  # 250000
    "sl_residual_layers": [],
    "sl_conv_layers": [],
    "sl_dense_layer_widths": [64, 64, 64],  # [128], [512]
    "sl_clip_low_prob": 0.0,  # 0.0 <- 16
    "per_alpha": 0.0,
    "per_beta": 0.0,
    "per_beta_final": 0.0,
    "per_epsilon": 0.00001,
    "n_step": 1,
    "atom_size": 1,
    "dueling": False,
    "clipnorm": 10.0,
    "sl_clipnorm": 10.0,
}
# config = NFSPConfig(config_dict=config_dict, game_config=MississippiMarblesConfig(), rl_config_type=RainbowConfig)
config = NFSPDQNConfig(
    config_dict=config_dict,
    game_config=LeducHoldemConfig(),
)

config.save_intermediate_weights = True

Using default save_intermediate_weights     : False
Using         training_steps                : 750000.0
Using default adam_epsilon                  : 1e-06
Using         learning_rate                 : 0.1
Using         clipnorm                      : 10.0
Using         optimizer                     : <class 'torch.optim.sgd.SGD'>
Using default weight_decay                  : 0.0
Using         loss_function                 : <utils.utils.MSELoss object at 0x1054d9e10>
Using default activation                    : relu
Using         kernel_initializer            : None
Using         minibatch_size                : 128
Using         replay_buffer_size            : 200000
Using         min_replay_buffer_size        : 128
Using         num_minibatches               : 2
Using default training_iterations           : 1
NFSPDQNConfig
Using         num_players                   : 2
Using default save_intermediate_weights     : False
Using         training_steps                : 750000.0
Usin

In [ ]:
from pathlib import Path
from nfsp_agent_clean import NFSPDQN
from agent_configs import NFSPDQNConfig, RainbowConfig
from game_configs import MississippiMarblesConfig, LeducHoldemConfig, TicTacToeConfig
from utils import KLDivergenceLoss, CategoricalCrossentropyLoss, HuberLoss, MSELoss
from torch.optim import Adam, SGD

# TODO: 8, 9, 10, 11, 12
# DONE: 14, 13
dir = "./checkpoints/NFSP-TicTacToe-16"
config = NFSPDQNConfig.load(Path(dir, "configs/config.yaml"))

In [2]:
import custom_gym_envs
import gymnasium as gym
from gymnasium.wrappers import FrameStack

# env = gym.make('custom_gym_envs/MississippiMarbles-v0', render_mode="human", players=2)
env = gym.make("custom_gym_envs/LeducHoldem-v0", encode_player_turn=False)
# env = FrameStack(env, 3)
# env = gym.make('custom_gym_envs/TicTacToe-v0', render_mode="rgb_array", player_turn_as_plane=False)
agent = NFSPDQN(env, config, name="NFSP-LeducHoldem", device="cpu")

observation_dimensions:  (36,)
num_actions:  4
observation_dimensions:  (36,)
num_actions:  4
int8
observation_dimensions:  (36,)
num_actions:  4
int8
observation_dimensions:  (36,)
num_actions:  4
(2000000, 36)
int8
observation_dimensions:  (36,)
num_actions:  4
(2000000, 36)
int8


/opt/homebrew/lib/python3.10/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.lz4_compress to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.lz4_compress` for environment variables or `env.get_wrapper_attr('lz4_compress')` that will search the reminding wrappers.
  logger.warn(


In [ ]:
dir = "./checkpoints/NFSP-LeducHoldem"
agent.load_from_checkpoint(dir, 44000)

In [ ]:
agent.checkpoint_interval = 1000
agent.checkpoint_trials = 1000
agent.train()

In [ ]:
agent.save_to_checkpoint(dir, 500000)

In [ ]:
import numpy as np
import torch

b = np.array([[1, 0, 1, 0, 1], [0, 1, 0, 1, 0], [1, 2, 3, 4, 5]])
print(np.sum(b == b.max(axis=1, keepdims=True), axis=1) > 1)
for c in range(len(b)):
    print(np.random.choice(np.where(b[c] == b[c].max())[0]))
arr = np.apply_along_axis(lambda x: np.random.choice(np.where(x == x.max())[0]), 1, b)
print(arr)
arr = torch.stack(
    [torch.tensor(np.random.choice(np.where(x == x.max())[0])) for x in b]
)
print(arr)

In [ ]:
a = agent.rl_agents[0].replay_buffer.action_buffer
s = agent.rl_agents[0].replay_buffer.observation_buffer
ns = agent.rl_agents[0].replay_buffer.next_observation_buffer
d = agent.rl_agents[0].replay_buffer.done_buffer
r = agent.rl_agents[0].replay_buffer.reward_buffer
print("actions", a)
print("states", s)
print("next states", ns)
print("dones", d)
print("rewards", r)

samples = agent.rl_agents[0].replay_buffer.sample()
print(samples)

In [ ]:
samples = agent.sl_agents[0].replay_buffer.sample()
print(samples)

In [ ]:
from pathlib import Path
from agent_configs import NFSPDQNConfig
from nfsp_agent_clean import NFSPDQN
import custom_gym_envs
import gymnasium as gym

# TODO: 8, 9, 10, 11, 12
# DONE: 14, 13
dir = "./checkpoints/NFSP-TicTacToe-32"
config = NFSPDQNConfig.load(Path(dir, "configs/config.yaml"))

env = gym.make(
    "custom_gym_envs/TicTacToe-v0", render_mode="human", encode_player_turn=True
)
# env = FrameStack(env, 3)

agent = NFSPDQN(env, config, name="NFSP-TicTacToe", device="cpu")

agent.load_from_checkpoint(
    dir, 499999
)  # test 104000 # 105000 # 87000 # 75000-83000 # 117000 # 118000 # 411000

# 38000 for nfsp tictactoe 26 and 40000

In [ ]:
print(agent.stats)

In [ ]:
from utils.utils import action_mask


env = gym.make(
    "custom_gym_envs/TicTacToe-v0", render_mode="human", encode_player_turn=True
)
# env = FrameStack(env, 3)
state, info = env.reset()
player = 0
agent_player = 1
done = False
agent.policies = ["average_strategy", "average_strategy"]
while not done:
    print(f"Player {player}")
    if player % 2 == agent_player:
        prediction = agent.predict(state, info)
        print(
            "Prediction",
            (
                action_mask(
                    (prediction * agent.rl_agents[0].support).sum(-1, keepdim=False),
                    [info["legal_moves"]],
                    mask_value=float("-inf"),
                )
                if agent.policies[agent_player] == "best_response"
                else prediction
            ),
        )
        action = agent.select_actions(prediction, info).item()
    else:
        action = int(input("Enter action: "))
        # prediction = agent.predict(state, info)
        # print(
        #     "Prediction",
        #     (
        #         action_mask(
        #             (prediction * agent.rl_agents[0].support).sum(-1, keepdim=False),
        #             [info["legal_moves"]],
        #             mask_value=float("-inf"),
        #         )
        #         if agent.policies[agent_player] == "best_response"
        #         else prediction
        #     ),
        # )
        # action = agent.select_actions(prediction, info).item()
    state, reward, terminated, truncated, info = env.step(action)
    print(action)
    player = (player + 1) % 2
    env.render()
    done = terminated or truncated

In [ ]:
agent.rl_agents[0].replay_buffer.sample()

In [ ]:
import numpy as np

test = np.array([[[1, 2, 3], [4, 5, 6]], [[7, 8, 9], [10, 11, 12]]])
probs = np.array([[[10, 20, 30], [40, 50, 60]], [[70, 80, 90], [100, 110, 120]]])
dones = np.array([False, True])
test[~dones] = probs[~dones]
print(test)

In [ ]:
from pathlib import Path
import yaml
from utils import plot_comparisons
from agent_configs import NFSPDQNConfig
from nfsp_agent_clean import NFSPDQN
import gymnasium as gym
import custom_gym_envs

stats_list = []

env = gym.make("custom_gym_envs/LeducHoldem-v0", encode_player_turn=False)

dir = "./checkpoints/NFSPDQN-LeducHoldem-7"
config = NFSPDQNConfig.load(Path(dir, "configs/config.yaml"))
agent = NFSPDQN(env, config, name="NFSPDQN-LeducHoldem", device="cpu")
agent.load_from_checkpoint(dir, 525000)
stats_list.append(agent.stats)

env = gym.make("custom_gym_envs/LeducHoldem-v0", encode_player_turn=True)
dir = "./checkpoints/NFSPDQN-LeducHoldem-8"
config = NFSPDQNConfig.load(Path(dir, "configs/config.yaml"))
agent = NFSPDQN(env, config, name="NFSPDQN-LeducHoldem", device="cpu")
agent.load_from_checkpoint(dir, 250000)
stats_list.append(agent.stats)

dir = "./checkpoints/NFSPDQN-LeducHoldem-9"
config = NFSPDQNConfig.load(Path(dir, "configs/config.yaml"))
agent = NFSPDQN(env, config, name="NFSPDQN-LeducHoldem", device="cpu")
agent.load_from_checkpoint(dir, 250000)
stats_list.append(agent.stats)

dir = "./checkpoints/NFSPDQN-LeducHoldem-10"
config = NFSPDQNConfig.load(Path(dir, "configs/config.yaml"))
agent = NFSPDQN(env, config, name="NFSPDQN-LeducHoldem", device="cpu")
agent.load_from_checkpoint(dir, 275000)
stats_list.append(agent.stats)

dir = "./checkpoints/NFSPDQN-LeducHoldem-11"
config = NFSPDQNConfig.load(Path(dir, "configs/config.yaml"))
agent = NFSPDQN(env, config, name="NFSPDQN-LeducHoldem", device="cpu")
agent.load_from_checkpoint(dir, 525000)
stats_list.append(agent.stats)

dir = "./checkpoints/NFSPDQN-LeducHoldem-12"
config = NFSPDQNConfig.load(Path(dir, "configs/config.yaml"))
agent = NFSPDQN(env, config, name="NFSPDQN-LeducHoldem", device="cpu")
agent.load_from_checkpoint(dir, 250000)
stats_list.append(agent.stats)

dir = "./checkpoints/NFSPDQN-LeducHoldem-13"
config = NFSPDQNConfig.load(Path(dir, "configs/config.yaml"))
agent = NFSPDQN(env, config, name="NFSPDQN-LeducHoldem", device="cpu")
agent.load_from_checkpoint(dir, 85000)
stats_list.append(agent.stats)


dir = "./checkpoints/NFSPDQN-LeducHoldem"
config = NFSPDQNConfig.load(Path(dir, "configs/config.yaml"))
agent = NFSPDQN(env, config, name="NFSPDQN-LeducHoldem", device="cpu")
agent.load_from_checkpoint(dir, 685000)
stats_list.append(agent.stats)

In [ ]:
# indices:
# 0: Default
# 1: Default Shared
# 2: PER + Shared
# 3: Dueling + PER + Shared
# 4: Distributional + Dueling + PER + Shared
# 5: Distributional + Dueling + PER + Shared + LR 0.05
plot_comparisons(stats_list, "NFSPDQN-LeducHoldem")

In [ ]:
from agent_configs import NFSPDQNConfig
from nfsp_agent_clean import NFSPDQN
import gymnasium as gym
import custom_gym_envs
from pathlib import Path

# the test agent
env = gym.make("custom_gym_envs/LeducHoldem-v0", encode_player_turn=True)
dir = "./checkpoints/NFSPDQN-LeducHoldem"
config = NFSPDQNConfig.load(Path(dir, "configs/config.yaml"))
test_agent = NFSPDQN(env, config, name="NFSPDQN-LeducHoldem", device="cpu")
test_agent.load_from_checkpoint(dir, 750000)

env = gym.make("custom_gym_envs/LeducHoldem-v0", encode_player_turn=True)
dir = "./checkpoints/NFSPDQN-LeducHoldem"
config = NFSPDQNConfig.load(Path(dir, "configs/config.yaml"))
challenger_agent = NFSPDQN(env, config, name="NFSPDQN-LeducHoldem", device="cpu")
challenger_agent.load_from_checkpoint(dir, 750000)

test_agent.policies = ["average_strategy", "average_strategy"]
# the challenger agent
challenger_agent.policies = ["best_response", "best_response"]

In [ ]:
import copy


test_player = 0
score = 0
test_score = 0
env = gym.make("custom_gym_envs/LeducHoldem-v0", encode_player_turn=True)
for _ in range(5000):
    print("Trial ", _)
    state, info = env.reset()
    done = False
    while not done:
        for player in range(2):
            if player == 0:
                prediction = test_agent.predict(state, info)
                action = test_agent.select_actions(prediction, info).item()
            else:
                prediction = challenger_agent.predict(state, info)
                action = challenger_agent.select_actions(prediction, info).item()
            print("Prediction", prediction)
            action_string = (
                "call"
                if action == 0
                else ("raise" if action == 1 else "fold" if action == 2 else "check")
            )
            print(action_string)
            next_state, reward, terminated, truncated, info = env.step(action)
            done = terminated or truncated
            state = next_state
            average_strategy_reward = reward[test_player]
            total_reward = sum(reward)
            test_score += total_reward - average_strategy_reward
            if done:
                break
score = test_score / 5000  #


test_player = 1
test_score = 0

env = gym.make("custom_gym_envs/LeducHoldem-v0", encode_player_turn=True)
for _ in range(5000):
    print("Trial ", _)
    state, info = env.reset()
    done = False
    while not done:
        for player in range(2):
            if player == 0:
                prediction = challenger_agent.predict(state, info)
                action = challenger_agent.select_actions(prediction, info).item()
            else:
                prediction = test_agent.predict(state, info)
                action = test_agent.select_actions(prediction, info).item()
            print("Prediction", prediction)
            action_string = (
                "call"
                if action == 0
                else ("raise" if action == 1 else "fold" if action == 2 else "check")
            )
            print(action_string)
            next_state, reward, terminated, truncated, info = env.step(action)
            done = terminated or truncated
            state = next_state
            average_strategy_reward = reward[test_player]
            total_reward = sum(reward)
            test_score += total_reward - average_strategy_reward
            if done:
                break
score += test_score / 5000  #
print(score)

In [ ]:
player_1 = agent.nfsp_agents[0]
player_2 = agent.nfsp_agents[1]
player_1.policy = "best_response"
state, info = env.reset()
print(state)
prediction = player_1.predict(state, info)
action = player_1.select_actions(prediction, info).item()
state, reward, terminated, truncated, info = env.step(action)
print(action)
print(state)


state, reward, terminated, truncated, info = env.step(4)
print(state)


prediction = player_1.predict(state, info)
action = player_1.select_actions(prediction, info).item()
state, reward, terminated, truncated, info = env.step(action)
print(action)
print(state)


state, reward, terminated, truncated, info = env.step(7)
print(state)


prediction = player_1.predict(state, info)
action = player_1.select_actions(prediction, info).item()
state, reward, terminated, truncated, info = env.step(action)
print(action)
print(state)


state, reward, terminated, truncated, info = env.step(3)
print(state)
print(terminated)


prediction = player_1.predict(state, info)
action = player_1.select_actions(prediction, info).item()
state, reward, terminated, truncated, info = env.step(action)
print(action)
print(state)

In [ ]:
state, info = env.reset()
print(state)
state_2, reward, terminated, truncated, info = env.step(0)
print(state_2)
print(state)

In [ ]:
samples = agent.nfsp_agents[0].rl_agent.replay_buffer.sample()
print(samples)

In [ ]:
import torch

q_values = torch.tensor(
    [
        [1, 0, 0, 0.5, -1],
        [-1, 1, 1, 1, -1],
    ]
)
legal_moves = [[0, 1, 3, 4], [2, 3, 4]]
mask = torch.zeros_like(q_values, dtype=torch.int8)
for i, legal in enumerate(legal_moves):
    mask[i, legal] = 1
print(mask)
q_values[mask == 0] = float("-inf")
selected_actions = q_values.argmax(1, keepdim=False)
print(q_values)